In [103]:
#Import libraries
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt

In [104]:
iShares_World = pd.read_csv("Data//iShares MCSI World Index Data.csv", index_col=0, parse_dates=True, dayfirst=True)
iShares_SNP500 = pd.read_csv("Data//iShares SNP500 Index Data.csv", index_col=0, parse_dates=True, dayfirst=True)
Vanguard_Global_Bond = pd.read_csv("Data//Vanguard Global Bond Data.csv",index_col=0, parse_dates=True, dayfirst=True)
BoE_Rate = pd.read_csv("Data//BoE Interest Rate Data.csv", index_col=0, parse_dates=True, dayfirst=True)



C:\Users\allan\AppData\Local\Temp\ipykernel_18152\3056266843.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  iShares_World = pd.read_csv("Data//iShares MCSI World Index Data.csv", index_col=0, parse_dates=True, dayfirst=True)
C:\Users\allan\AppData\Local\Temp\ipykernel_18152\3056266843.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  iShares_SNP500 = pd.read_csv("Data//iShares SNP500 Index Data.csv", index_col=0, parse_dates=True, dayfirst=True)
C:\Users\allan\AppData\Local\Temp\ipykernel_18152\3056266843.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  Vanguard_G

In [105]:
iShares_World = iShares_World.rename(columns={"Close": "iShares_World_Close"})
iShares_SNP500 = iShares_SNP500.rename(columns={"Close": "iShares_SNP500_Close"})
Vanguard_Global_Bond = Vanguard_Global_Bond.rename(columns={"Close": "Vanguard_Global_Bond_Close"})
BoE_Rate = BoE_Rate.rename(columns={"Rate": "BoE_Rate"})


In [106]:
iShares_World = iShares_World["iShares_World_Close"]
iShares_SNP500 = iShares_SNP500["iShares_SNP500_Close"]
Vanguard_Global_Bond = Vanguard_Global_Bond[ "Vanguard_Global_Bond_Close"]


In [107]:
merged = pd.merge(iShares_World,iShares_SNP500,on="Date")
merged = pd.merge(merged,Vanguard_Global_Bond,on="Date")
merged = pd.merge(merged,BoE_Rate,on="Date")

merged

,iShares_World_Close,iShares_SNP500_Close,Vanguard_Global_Bond_Close,BoE_Rate
Date,,,,
2025-07-01,"10,878.00","14,371.00",155.31,4.25
2025-06-01,"10,719.00","14,130.00",156.29,4.25
2025-05-01,"10,335.00","13,442.00",154.85,4.50
2025-04-01,"9,681.50","12,577.00",155.39,4.50
2025-03-01,"9,727.00","12,648.00",153.97,4.50
...,...,...,...,...
2014-05-01,"3,866.50","4,371.50",133.24,0.50
2014-04-01,"3,775.00","4,265.50",131.96,0.50
2014-03-01,"3,751.50","4,243.00",131.05,0.50


In [108]:
merged = merged.replace(',','', regex=True)
merged

,iShares_World_Close,iShares_SNP500_Close,Vanguard_Global_Bond_Close,BoE_Rate
Date,,,,
2025-07-01,10878.00,14371.00,155.31,4.25
2025-06-01,10719.00,14130.00,156.29,4.25
2025-05-01,10335.00,13442.00,154.85,4.50
2025-04-01,9681.50,12577.00,155.39,4.50
2025-03-01,9727.00,12648.00,153.97,4.50
...,...,...,...,...
2014-05-01,3866.50,4371.50,133.24,0.50
2014-04-01,3775.00,4265.50,131.96,0.50
2014-03-01,3751.50,4243.00,131.05,0.50


In [109]:
merged.dtypes


iShares_World_Close            object
iShares_SNP500_Close           object
Vanguard_Global_Bond_Close    float64
BoE_Rate                      float64
dtype: object

In [110]:
merged[["iShares_World_Close", "iShares_SNP500_Close"]]=merged[["iShares_World_Close", "iShares_SNP500_Close"]].astype(float)
merged.dtypes

iShares_World_Close           float64
iShares_SNP500_Close          float64
Vanguard_Global_Bond_Close    float64
BoE_Rate                      float64
dtype: object

In [111]:
date_range = pd.date_range('2015-01-01', '2024-12-31', freq='MS')
year_starts = pd.date_range('2015-01-01', '2024-12-31', freq='YS')

In [112]:
print(year_starts)

DatetimeIndex(['2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01',
               '2019-01-01', '2020-01-01', '2021-01-01', '2022-01-01',
               '2023-01-01', '2024-01-01'],
              dtype='datetime64[ns]', freq='AS-JAN')


In [113]:
def invest_monthly(iShares_World_prop,iShares_SNP500_prop,Vanguard_Global_Bond_prop,Cash_prop):

    #PROPORTIONS
    # iShares_World_prop = 0.3
    # iShares_SNP500_prop = 0.3
    # Vanguard_Global_Bond_prop = 0.3
    #Cash_prop = 0.1

    total_shares = 0
    total_invested = 0
    
    iShares_World_total_shares = 0
    iShares_SNP500_total_shares = 0
    Vanguard_Global_Bond_total_shares = 0
    Cash_total = 0
    
    df = pd.DataFrame({'Date': [], 'iShares_World_Value': [], 'iShares_SNP500_Value': [],
                       "Vanguard_Global_Bond_Value":[],"Cash_total":[], "Total_Value":[]}).set_index("Date")
    
    
    for date in date_range:
        #iShares World
        iShares_World_current_close = merged.loc[date,"iShares_World_Close"]
        iShares_World_total_shares =  iShares_World_total_shares + (100*iShares_World_prop/iShares_World_current_close)
        iShares_World_Value = iShares_World_total_shares*iShares_World_current_close
        
        #iShares SNP500
        iShares_SNP500_current_close = merged.loc[date,"iShares_SNP500_Close"]
        iShares_SNP500_total_shares =  iShares_SNP500_total_shares + (100*iShares_SNP500_prop/iShares_SNP500_current_close)
        iShares_SNP500_Value = iShares_SNP500_total_shares * iShares_SNP500_current_close
        
        #Vanguard
        Vanguard_Global_Bond_current_close = merged.loc[date,"Vanguard_Global_Bond_Close"]
        Vanguard_Global_Bond_total_shares =  Vanguard_Global_Bond_total_shares + (100*Vanguard_Global_Bond_prop/Vanguard_Global_Bond_current_close)
        Vanguard_Global_Bond_Value = Vanguard_Global_Bond_total_shares*Vanguard_Global_Bond_current_close
        
        #Cash
        BoE_current_rate = merged.loc[date,"BoE_Rate"]
        BoE_current_rate_monthly = BoE_current_rate/12
        Cash_total = (100*Cash_prop) + (BoE_current_rate_monthly*Cash_total/100) + Cash_total
        Total_Value=(iShares_World_total_shares*iShares_World_current_close + 
                     iShares_SNP500_total_shares*iShares_SNP500_current_close + 
                     Vanguard_Global_Bond_total_shares*Vanguard_Global_Bond_current_close + Cash_total)
    
        newrow = pd.DataFrame({'Date': [date], 'iShares_World_Value': [iShares_World_Value], 'iShares_SNP500_Value': [iShares_SNP500_Value],
        "Vanguard_Global_Bond_Value":[Vanguard_Global_Bond_Value],"Cash_total":[Cash_total], "Total_Value": [Total_Value]}).set_index("Date")
    
        df = pd.concat([df, newrow])
   
    
    results = df
    return results



In [114]:
def rebalancing_yearly(iShares_World_prop,iShares_SNP500_prop,Vanguard_Global_Bond_prop,Cash_prop):

    total_shares = 0
    total_invested = 0
    
    iShares_World_total_shares = 0
    iShares_SNP500_total_shares = 0
    Vanguard_Global_Bond_total_shares = 0
    Cash_total = 0
    Total_Value = 0
    
    df = pd.DataFrame({'Date': [], 'iShares_World_Value': [], 'iShares_SNP500_Value': [],
                       "Vanguard_Global_Bond_Value":[],"Cash_total":[], "Total_Value":[]}).set_index("Date")
    
    
    for date in date_range:
        if date in year_starts:
            #iShares World
            iShares_World_current_close = merged.loc[date,"iShares_World_Close"]
            iShares_World_total_shares =  (Total_Value+100)*iShares_World_prop/iShares_World_current_close
            iShares_World_Value = iShares_World_total_shares*iShares_World_current_close
            
            #iShares SNP500
            iShares_SNP500_current_close = merged.loc[date,"iShares_SNP500_Close"]
            iShares_SNP500_total_shares =  (Total_Value + 100)*iShares_SNP500_prop/iShares_SNP500_current_close
            iShares_SNP500_Value = iShares_SNP500_total_shares * iShares_SNP500_current_close
            
            #Vanguard
            Vanguard_Global_Bond_current_close = merged.loc[date,"Vanguard_Global_Bond_Close"]
            Vanguard_Global_Bond_total_shares = (Total_Value + 100)*Vanguard_Global_Bond_prop/Vanguard_Global_Bond_current_close
            Vanguard_Global_Bond_Value = Vanguard_Global_Bond_total_shares*Vanguard_Global_Bond_current_close
            
            #Cash
            BoE_current_rate = merged.loc[date,"BoE_Rate"]
            BoE_current_rate_monthly = BoE_current_rate/12
            Cash_total = (100+Total_Value)*Cash_prop

        else:
            #iShares World
            iShares_World_current_close = merged.loc[date,"iShares_World_Close"]
            iShares_World_total_shares =  iShares_World_total_shares + (100*iShares_World_prop/iShares_World_current_close)
            iShares_World_Value = iShares_World_total_shares*iShares_World_current_close
            
            #iShares SNP500
            iShares_SNP500_current_close = merged.loc[date,"iShares_SNP500_Close"]
            iShares_SNP500_total_shares =  iShares_SNP500_total_shares + (100*iShares_SNP500_prop/iShares_SNP500_current_close)
            iShares_SNP500_Value = iShares_SNP500_total_shares * iShares_SNP500_current_close
            
            #Vanguard
            Vanguard_Global_Bond_current_close = merged.loc[date,"Vanguard_Global_Bond_Close"]
            Vanguard_Global_Bond_total_shares =  Vanguard_Global_Bond_total_shares + (100*Vanguard_Global_Bond_prop/Vanguard_Global_Bond_current_close)
            Vanguard_Global_Bond_Value = Vanguard_Global_Bond_total_shares*Vanguard_Global_Bond_current_close
            
            #Cash
            BoE_current_rate = merged.loc[date,"BoE_Rate"]
            BoE_current_rate_monthly = BoE_current_rate/12
            Cash_total = (100*Cash_prop) + (BoE_current_rate_monthly*Cash_total/100) + Cash_total
        
        Total_Value=(iShares_World_total_shares*iShares_World_current_close + 
                         iShares_SNP500_total_shares*iShares_SNP500_current_close + 
                         Vanguard_Global_Bond_total_shares*Vanguard_Global_Bond_current_close + Cash_total)
        newrow = pd.DataFrame({'Date': [date], 'iShares_World_Value': [iShares_World_Value], 'iShares_SNP500_Value': [iShares_SNP500_Value],
            "Vanguard_Global_Bond_Value":[Vanguard_Global_Bond_Value],"Cash_total":[Cash_total], "Total_Value": [Total_Value]}).set_index("Date")        
        df = pd.concat([df, newrow])
   
    results = df
    return results



In [115]:
rebalancing_yearly(0.3,0.3,0.3,0.1).loc[year_starts]

,iShares_World_Value,iShares_SNP500_Value,Vanguard_Global_Bond_Value,Cash_total,Total_Value
2015-01-01,30.000000,30.000000,30.000000,10.000000,100.000000
2016-01-01,389.660234,389.660234,389.660234,129.886745,1298.867448
2017-01-01,806.870469,806.870469,806.870469,268.956823,2689.568231
2018-01-01,1276.766180,1276.766180,1276.766180,425.588727,4255.887266
2019-01-01,1534.652367,1534.652367,1534.652367,511.550789,5115.507890
2020-01-01,2101.234103,2101.234103,2101.234103,700.411368,7004.113678
2021-01-01,2678.986061,2678.986061,2678.986061,892.995354,8929.953536
2022-01-01,3468.997249,3468.997249,3468.997249,1156.332416,11563.324162
2023-01-01,3406.162870,3406.162870,3406.162870,1135.387623,11353.876234
2024-01-01,4183.290091,4183.290091,4183.290091,1394.430030,13944.300302


In [116]:
def invest_yearly(iShares_World_prop,iShares_SNP500_prop,Vanguard_Global_Bond_prop,Cash_prop):

    total_shares = 0
    total_invested = 0
    
    iShares_World_total_shares = 0
    iShares_SNP500_total_shares = 0
    Vanguard_Global_Bond_total_shares = 0
    Cash_total = 0
    Total_Value = 0
    
    df = pd.DataFrame({'Date': [], 'iShares_World_Value': [], 'iShares_SNP500_Value': [],
                       "Vanguard_Global_Bond_Value":[],"Cash_total":[], "Total_Value":[]}).set_index("Date")
    
    
    for date in date_range:
        if date in year_starts:
            #iShares World
            iShares_World_current_close = merged.loc[date,"iShares_World_Close"]
            iShares_World_total_shares =  iShares_World_total_shares + (1200*iShares_World_prop/iShares_World_current_close)
            iShares_World_Value = iShares_World_total_shares*iShares_World_current_close
            
            #iShares SNP500
            iShares_SNP500_current_close = merged.loc[date,"iShares_SNP500_Close"]
            iShares_SNP500_total_shares =  iShares_SNP500_total_shares + (1200*iShares_SNP500_prop/iShares_SNP500_current_close)
            iShares_SNP500_Value = iShares_SNP500_total_shares * iShares_SNP500_current_close
            
            #Vanguard Global Bond
            Vanguard_Global_Bond_current_close = merged.loc[date,"Vanguard_Global_Bond_Close"]
            Vanguard_Global_Bond_total_shares =  Vanguard_Global_Bond_total_shares + (1200*Vanguard_Global_Bond_prop/Vanguard_Global_Bond_current_close)
            Vanguard_Global_Bond_Value = Vanguard_Global_Bond_total_shares*Vanguard_Global_Bond_current_close
            
            #Cash
            BoE_current_rate = merged.loc[date,"BoE_Rate"]
            BoE_current_rate_monthly = BoE_current_rate/12
            Cash_total = (1200*Cash_prop) + (BoE_current_rate_monthly*Cash_total/100) + Cash_total
        else:
            #iShares World
            iShares_World_current_close = merged.loc[date,"iShares_World_Close"]
            iShares_World_Value = iShares_World_total_shares*iShares_World_current_close
            
            #iShares SNP500
            iShares_SNP500_current_close = merged.loc[date,"iShares_SNP500_Close"]
            iShares_SNP500_Value = iShares_SNP500_total_shares * iShares_SNP500_current_close
            
            #Vanguard Global Bond
            Vanguard_Global_Bond_current_close = merged.loc[date,"Vanguard_Global_Bond_Close"]
            Vanguard_Global_Bond_Value = Vanguard_Global_Bond_total_shares*Vanguard_Global_Bond_current_close
            
            #Cash
            BoE_current_rate = merged.loc[date,"BoE_Rate"]
            BoE_current_rate_monthly = BoE_current_rate/12
            Cash_total = (BoE_current_rate_monthly*Cash_total/100) + Cash_total            
        
        Total_Value=(iShares_World_total_shares*iShares_World_current_close + 
                         iShares_SNP500_total_shares*iShares_SNP500_current_close + 
                         Vanguard_Global_Bond_total_shares*Vanguard_Global_Bond_current_close + Cash_total)
        newrow = pd.DataFrame({'Date': [date], 'iShares_World_Value': [iShares_World_Value], 'iShares_SNP500_Value': [iShares_SNP500_Value],
                                "Vanguard_Global_Bond_Value":[Vanguard_Global_Bond_Value],"Cash_total":[Cash_total], 
                               "Total_Value": [Total_Value]}).set_index("Date")        
        df = pd.concat([df, newrow])
   
    results = df
    return results



In [117]:
Proportions_df = pd.DataFrame({"iShares_World_prop": [0.45, 0.6, 0.7,0.8], "Vanguard_Global_Bond_prop": [0.45, 0.3, 0.2, 0.1], "Cash_prop":[0.1,0.1,0.1, 0.1]})

In [118]:
Proportions_df

,iShares_World_prop,Vanguard_Global_Bond_prop,Cash_prop
0,0.45,0.45,0.1
1,0.60,0.30,0.1
2,0.70,0.20,0.1
3,0.80,0.10,0.1


In [119]:
first_row_data = invest_monthly(Proportions_df.iloc[0][0], 0, Proportions_df.iloc[0][1], Proportions_df.iloc[0][2])
results_df = pd.DataFrame(first_row_data.tail(1))


for i in range(1, len(Proportions_df)):
    data = invest_monthly(Proportions_df.iloc[i][0], 0, Proportions_df.iloc[i][1], Proportions_df.iloc[i][2])
    newrow = data.tail(1)


    results_df = pd.concat([results_df, newrow])

results_df = results_df.drop(columns =["iShares_SNP500_Value"])
results_df = results_df.reset_index(drop=True)
invest_monthly_results = pd.merge(results_df, Proportions_df, left_index=True, right_index=True)



In [120]:
first_row_data = rebalancing_yearly(Proportions_df.iloc[0][0], 0, Proportions_df.iloc[0][1], Proportions_df.iloc[0][2])
results_df = pd.DataFrame(first_row_data.tail(1))


for i in range(1, len(Proportions_df)):
    data = rebalancing_yearly(Proportions_df.iloc[i][0], 0, Proportions_df.iloc[i][1], Proportions_df.iloc[i][2])
    newrow = data.tail(1)


    results_df = pd.concat([results_df, newrow])

results_df = results_df.drop(columns =["iShares_SNP500_Value"])
results_df = results_df.reset_index(drop=True)
yearly_rebalance_results = pd.merge(results_df, Proportions_df, left_index=True, right_index=True)

In [121]:
print(yearly_rebalance_results)

   iShares_World_Value  Vanguard_Global_Bond_Value   Cash_total   Total_Value  \
0          7409.474479                 6441.309965  1461.863225  15312.647669   
1         10443.703741                 4537.741737  1544.861954  16526.307432   
2         12640.091526                 3137.538639  1602.310685  17379.940849   
3         14982.175580                 1626.627510  1661.466302  18270.269391   

   iShares_World_prop  Vanguard_Global_Bond_prop  Cash_prop  
0                0.45                       0.45        0.1  
1                0.60                       0.30        0.1  
2                0.70                       0.20        0.1  
3                0.80                       0.10        0.1  


In [122]:
yearly_rebalance_results.to_csv("Exported_Data//Yearly Rebalanced Results.csv")
invest_monthly_results.to_csv("Exported_Data//Invest Monthly Results.csv")

In [123]:
merged = merged.loc[date_range]

In [124]:
merged = merged.drop(columns = ["BoE_Rate"])


In [126]:
normalised_dataset = pd.DataFrame([])

for column in ["iShares_World_Close",	"iShares_SNP500_Close",	"Vanguard_Global_Bond_Close"]:
    normalised_dataset[column] = merged[column] / merged[column][0]


In [128]:
normalised_dataset.to_csv("Exported_Data//Normalised Diverse Portfolio.csv")

In [129]:
normalised_dataset

,iShares_World_Close,iShares_SNP500_Close,Vanguard_Global_Bond_Close
2015-01-01,1.000000,1.000000,1.000000
2015-02-01,1.057080,1.050873,0.994339
2015-03-01,1.056095,1.039918,1.000637
2015-04-01,1.066921,1.047836,0.995118
2015-05-01,1.074302,1.052392,0.990094
...,...,...,...
2024-08-01,2.377906,2.750841,1.078186
2024-09-01,2.416041,2.819069,1.091134
2024-10-01,2.409152,2.817008,1.073233
2024-11-01,2.521835,2.968652,1.087384
